<a href="https://colab.research.google.com/github/shivammehta007/NLPResearch/blob/master/Comparison_of_NLP_Models_IMDB_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# We will compare NLP Models with IMDB Classification

## Imports

In [21]:
import os
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext
from torchtext import datasets, data
import numpy as np
import pandas as pd

from pprint import pprint

!pip install -U tqdm
# Need to restart runtime on google collab for it to work
from tqdm.notebook import tqdm

Requirement already up-to-date: tqdm in /usr/local/lib/python3.6/dist-packages (4.42.0)


## Seed Set

In [0]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
random.seed(SEED)

# Load the Data

## TorchText Data

Initialize two Field and Label 

In [0]:
TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype = torch.float)

In [0]:
%%time
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [30]:
print(len(train_data), len(test_data))
print(vars(train_data.examples[0]))

25000 25000
{'text': ['This', 'is', 'an', 'absurdist', 'dark', 'comedy', 'from', 'Belgium', '.', 'Shot', 'perfectly', 'in', 'crisp', 'black', 'and', 'white', ',', 'Benoît', 'Poelvoorde', '(', 'Man', 'Bites', 'Dog', ')', 'is', 'on', 'fine', 'form', 'as', 'Roger', ',', 'the', 'angry', ',', 'obsessive', 'father', 'of', 'a', 'family', 'in', 'a', 'small', ',', 'sullen', 'Belgian', 'mining', 'town', '.', 'Roger', 'is', 'a', 'photographer', 'who', ',', 'along', 'with', 'his', 'young', 'daughter', 'Luise', ',', 'visits', 'road', 'accidents', 'to', 'take', 'photos', '.', 'He', 'is', 'also', 'obsessed', 'with', 'winning', 'a', 'car', 'by', 'entering', 'a', 'competition', 'where', 'the', 'contestant', 'has', 'to', 'break', 'a', 'record', '-', 'and', 'he', 'decides', 'that', 'his', 'son', ',', 'Michel', ',', 'must', 'attempt', 'to', 'break', 'the', 'record', 'of', 'perpetually', 'walking', 'through', 'a', 'door', '-', 'he', 'even', 'hires', 'an', 'overweight', 'coach', 'to', 'train', 'him', '.', '

In [0]:
train_data, valid_data = train_data.split(split_ratio=0.8, random_state=random.seed(SEED))

In [38]:
print(f'Train Data: \t\t {len(train_data):,}')
print(f'Validation Dataset: \t {len(valid_data):,}')
print(f'Test Data: \t\t {len(test_data):,}')

Train Data: 		 20,000
Validation Dataset: 	 5,000
Test Data: 		 25,000


## Numpy Data
We Converted Data to Numpy for implementation of Classical Algorithms

In [42]:
type(train_data)

torchtext.data.dataset.Dataset

In [0]:
def get_dataframe_from_dataset(dataset, labels = {'X': 'text', 'y': 'label'}):
    """Utility Method to convert torchext.data.Dataset to numpy array of text and label"""
    i = 0
    data = {'X' : [], 'y' : []}
    for example in tqdm(dataset):
        text = ' '.join(list(example.text))
        data['X'].append(text)
        data['y'].append(example.label)
    
    assert len(data['X']) == len(data['y'])

    return pd.DataFrame(data).rename(columns=labels)


In [127]:
train_df = get_dataframe_from_dataset(train_data)
val_df = get_dataframe_from_dataset(valid_data)
test_df = get_dataframe_from_dataset(test_data)

In [128]:
test_df.head()

,text,label
0,Hehehe . This was one of the best funny road m...,pos
1,This movie is good for entertainment purposes ...,pos
2,Corniness Warning . As many fellow IMDb users ...,pos
3,This movie starts off somewhat slowly and gets...,pos
4,I got some free tickets via the Times to see t...,pos


# Implementation

## Classical ML algorithms